In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Lreg').getOrCreate()
from pyspark.ml.regression import LinearRegression

In [2]:
#reading test file
training = spark.read.format('libsvm').load('/FileStore/tables/sample_linear_regression_data.txt')

In [3]:
training.show()

+-------------------+--------------------+
 label| features|
+-------------------+--------------------+
 -9.490009878824548|(10,[0,1,2,3,4,5,...|
 0.2577820163584905|(10,[0,1,2,3,4,5,...|
 -4.438869807456516|(10,[0,1,2,3,4,5,...|
-19.782762789614537|(10,[0,1,2,3,4,5,...|
 -7.966593841555266|(10,[0,1,2,3,4,5,...|
 -7.896274316726144|(10,[0,1,2,3,4,5,...|
 -8.464803554195287|(10,[0,1,2,3,4,5,...|
 2.1214592666251364|(10,[0,1,2,3,4,5,...|
 1.0720117616524107|(10,[0,1,2,3,4,5,...|
-13.772441561702871|(10,[0,1,2,3,4,5,...|
 -5.082010756207233|(10,[0,1,2,3,4,5,...|
 7.887786536531237|(10,[0,1,2,3,4,5,...|
 14.323146365332388|(10,[0,1,2,3,4,5,...|
-20.057482615789212|(10,[0,1,2,3,4,5,...|
-0.8995693247765151|(10,[0,1,2,3,4,5,...|
 -19.16829262296376|(10,[0,1,2,3,4,5,...|
 5.601801561245534|(10,[0,1,2,3,4,5,...|
-3.2256352187273354|(10,[0,1,2,3,4,5,...|
 1.5299675726687754|(10,[0,1,2,3,4,5,...|
 -0.250102447941961|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 20 rows

In [4]:
#build the model
lr = LinearRegression(featuresCol='features',labelCol='label',predictionCol='prediction')

In [5]:
#train the model
LrModel = lr.fit(training)

In [6]:
LrModel.coefficients

Out[ 13 ]: DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [7]:
LrModel.intercept

Out[ 14 ]: 0.14228558260358093

In [8]:
summary =  LrModel.summary

In [9]:
summary.r2

Out[ 26 ]: 0.027839179518600154

In [10]:
summary.meanAbsoluteError

Out[ 27 ]: 8.145215527783876

In [11]:
summary.rootMeanSquaredError

Out[ 28 ]: 10.16309157133015

In [12]:
all_data = spark.read.format('libsvm').load('/FileStore/tables/sample_linear_regression_data.txt')

In [13]:
train_data ,test_data = all_data.randomSplit([0.7,0.3])

In [14]:
train_data.describe()

Out[ 32 ]: DataFrame[summary: string, label: string]

In [15]:
train_data.show()

+-------------------+--------------------+
 label| features|
+-------------------+--------------------+
-28.046018037776633|(10,[0,1,2,3,4,5,...|
-26.736207182601724|(10,[0,1,2,3,4,5,...|
-22.949825936196074|(10,[0,1,2,3,4,5,...|
-22.837460416919342|(10,[0,1,2,3,4,5,...|
-20.212077258958672|(10,[0,1,2,3,4,5,...|
-20.057482615789212|(10,[0,1,2,3,4,5,...|
-19.884560774273424|(10,[0,1,2,3,4,5,...|
-19.872991038068406|(10,[0,1,2,3,4,5,...|
-19.782762789614537|(10,[0,1,2,3,4,5,...|
-19.402336030214553|(10,[0,1,2,3,4,5,...|
 -19.16829262296376|(10,[0,1,2,3,4,5,...|
-18.845922472898582|(10,[0,1,2,3,4,5,...|
-17.803626188664516|(10,[0,1,2,3,4,5,...|
 -17.32672073267595|(10,[0,1,2,3,4,5,...|
-17.065399625876015|(10,[0,1,2,3,4,5,...|
-17.026492264209548|(10,[0,1,2,3,4,5,...|
 -16.71909683360509|(10,[0,1,2,3,4,5,...|
 -16.08565904102149|(10,[0,1,2,3,4,5,...|
-15.951512565794573|(10,[0,1,2,3,4,5,...|
-15.780685032623301|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 20 rows

In [16]:
train_data.describe().show()

+-------+-------------------+
summary| label|
+-------+-------------------+
 count| 340|
 mean| 0.9427050943411365|
 stddev| 10.319136913533418|
 min|-28.046018037776633|
 max| 27.78383192005107|
+-------+-------------------+

In [17]:
test_data.describe().show()

+-------+-------------------+
summary| label|
+-------+-------------------+
 count| 161|
 mean| -1.191418833268928|
 stddev| 10.196035017129976|
 min|-28.571478869743427|
 max| 22.647750304177556|
+-------+-------------------+

In [18]:
correct_model = lr.fit(train_data)

In [19]:
test_results = correct_model.evaluate(test_data)

In [20]:
test_results.r2

Out[ 41 ]: -0.06159627977446691

In [21]:
test_results.rootMeanSquaredError

Out[ 42 ]: 10.472685595589864

In [22]:
#create unlabeled data for predictions
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
 features|
+--------------------+
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows

In [23]:
predictions = correct_model.transform(unlabeled_data)

In [24]:
predictions.show()

+--------------------+--------------------+
 features| prediction|
+--------------------+--------------------+
(10,[0,1,2,3,4,5,...| -0.3761550449068779|
(10,[0,1,2,3,4,5,...| 0.3126641155482716|
(10,[0,1,2,3,4,5,...| 1.045389032942147|
(10,[0,1,2,3,4,5,...| -0.1005852005862945|
(10,[0,1,2,3,4,5,...| 2.025951024526325|
(10,[0,1,2,3,4,5,...|-0.16440177224174157|
(10,[0,1,2,3,4,5,...| 1.8556084104797552|
(10,[0,1,2,3,4,5,...| -1.713079006732288|
(10,[0,1,2,3,4,5,...| 1.5965115794850497|
(10,[0,1,2,3,4,5,...| 3.499657731797017|
(10,[0,1,2,3,4,5,...| 2.8437977079004293|
(10,[0,1,2,3,4,5,...| 0.3507855759774009|
(10,[0,1,2,3,4,5,...| 1.3699395341715563|
(10,[0,1,2,3,4,5,...| 1.3294564559374644|
(10,[0,1,2,3,4,5,...| 3.866727951875848|
(10,[0,1,2,3,4,5,...| -1.8581985846352151|
(10,[0,1,2,3,4,5,...| 0.22074959629857638|
(10,[0,1,2,3,4,5,...| 1.7113254014650157|
(10,[0,1,2,3,4,5,...| 1.419358054143895|
(10,[0,1,2,3,4,5,...| 0.9556425968755342|
+--------------------+--------------------+
only showing top 20 rows